导入库

In [1]:
import datetime as dt
import gc
import logging

import akshare as ak
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns
from dateutil import relativedelta as rd
from joblib import Parallel, delayed
from matplotlib import pyplot as plt
from plotly.subplots import make_subplots
from pricelib import StandardSnowball, set_evaluation_date
from scipy import optimize
from tqdm.auto import tqdm, trange
from tqdm.contrib import tzip

pd.set_option("display.max_columns", None)

plt.rcParams["font.sans-serif"] = "SimHei"
plt.rcParams["axes.unicode_minus"] = False

logging.basicConfig(level=logging.INFO)

%config InlineBackend.figure_format = 'retina'

# 获取收盘数据

In [2]:
def get_price(code, type) -> pd.Series:
    """_summary_

    Args:
        code (str): 指数/ETF代码
        type (str): 代码类型

    Returns:
        pd.Series: 指数收盘价
    """
    # 用akshare获取指数量价数据
    if type == "index":
        price_df = ak.stock_zh_index_daily(symbol=code)
    # 用akshare获取ETF量价数据
    elif type == "etf":
        price_df = ak.fund_etf_hist_sina(symbol=code)
    elif type == "stock_A":
        price_df = ak.stock_zh_a_hist(
            symbol=code,
            period="daily",
            start_date="20100101",
            end_date=dt.datetime.now().strftime("%Y%m%d"),
            adjust="",
        )
        price_df.rename(columns={"日期": "date", "收盘": "close"}, inplace=True)
    elif type == "stock_H":
        price_df = ak.stock_hk_hist(
            symbol=code,
            period="daily",
            start_date="20100101",
            end_date=dt.datetime.now().strftime("%Y%m%d"),
            adjust="",
        )
        price_df.rename(columns={"日期": "date", "收盘": "close"}, inplace=True)
    # 将str格式的日期转换为pd.timestamp
    price_df.set_index("date", inplace=True)
    # 将日期设为index
    price_df.index = pd.to_datetime(price_df.index)
    # 选取close列
    price_df = price_df.loc[:, "close"].astype(float)

    return price_df

# 历史波动率

绘制波动率序列和波动率锥

In [3]:
index_df = get_price(code="sh000905", type="index")

# 计算不同滚动窗口的波动率
vol_dfs = []
for i in [5, 10, 21, 42, 63, 84, 105, 126, 252]:
    vol_dfs.append(
        index_df.loc[dt.datetime(2005, 1, 1) : dt.datetime.now()]
        .pct_change(fill_method=None)
        .rolling(i)
        .std()
        .dropna()
        * np.sqrt(252)
    )
vol_dfs = pd.concat(vol_dfs, axis=1)
vol_dfs.columns = [5, 10, 21, 42, 63, 84, 105, 126, 252]

# 计算每个滑动窗口的波动率的历史分位数（最大、80%、60%、40%、20%、最小）
quantiles = [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1]
vol_selected = vol_dfs.loc[dt.datetime.now() - rd.relativedelta(years=15) :, :252]
vol_quantiles = vol_selected.quantile(quantiles)
vol_quantiles = vol_quantiles.T
vol_quantiles.columns = [f"{int(i*100)}%" for i in quantiles]

# 创建画布
fig = make_subplots(
    rows=2,
    cols=1,
    subplot_titles=[
        "历史波动率序列",
        "历史波动率锥",
    ],
)

# 绘制历史波动率
for period in [21, 63, 126]:
    vol_fig = vol_dfs.loc[dt.datetime.now() - rd.relativedelta(years=15) :, period]
    fig.add_trace(
        go.Scatter(
            x=vol_fig.index,
            y=vol_fig,
            mode="lines",
            name=f"HV{period}",
            legendgroup="1",
        ),
        row=1,
        col=1,
    )

# 添加波动率锥
for col in vol_quantiles.columns:
    fig.add_trace(
        go.Scatter(
            x=vol_quantiles.index,
            y=vol_quantiles[col],
            mode="lines",
            name=col,
            legendgroup="2",
        ),
        row=2,
        col=1,
    )

# 添加最新波动率到波动率锥中
fig.add_trace(
    go.Scatter(
        x=vol_selected.columns,
        y=vol_selected.iloc[-1, :],
        mode="lines+markers",
        name="Latest Vol",
        legendgroup="2",
    ),
    row=2,
    col=1,
)

fig["layout"]["xaxis"].update(
    {
        "title": "日期",
        "tickformat": "%Y-%m-%d",
        "rangeslider": dict(visible=True, thickness=0.05),
        "rangeselector": dict(
            buttons=[
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all"),
            ]
        ),
    }
)
fig["layout"]["yaxis"].update({"title": "波动率", "tickformat": ".2%"})
fig["layout"]["xaxis2"].update(
    {
        "title": "窗口",
        "tickvals": [5, 10, 21, 42, 63, 84, 105, 126, 252],
        "ticktext": ["1W", "2W", "1M", "2M", "3M", "4M", "5M", "6M", "1Y"],
    }
)
fig["layout"]["yaxis2"].update({"title": "波动率", "tickformat": ".2%"})
fig.update_layout(
    template="plotly_white",
    width=900,
    height=1000,
    hovermode="x unified",  # 设置鼠标悬停模式
    legend_tracegroupgap=460,
)
# fig.update_xaxes(rangeslider_visible=True)
fig.show()

# 净值化指数雪球参数研究

## 净值化雪球回测

### 工具函数

获取敲出观察日

In [4]:
def get_obs_dates(start_date, end_date, lock_term) -> list:
    n_dates = (end_date - start_date).days + 1
    n_obs = round(n_dates / 30) + 1
    obs_dates = [
        calendar[calendar >= start_date + rd.relativedelta(months=i)][0]
        .to_pydatetime()
        .date()
        for i in range(n_obs)
    ][lock_term:]
    if obs_dates[-1] > end_date.date():
        obs_dates[-1] = calendar[calendar <= end_date][-1].to_pydatetime().date()
    return obs_dates


使用理论票息的期权对象

In [5]:
def get_option(
    r,
    q,
    vol,
    s0,
    coupon,
    start_date,
    end_date,
    obs_dates,
    lock_term,
    ko,
    ki,
    back_calc=True,
) -> StandardSnowball:
    # 暂时停止logging
    logging.disable(logging.INFO)

    if back_calc is True:
        # 定义函数，用于计算理论票息
        def coupon2price(c):
            set_evaluation_date(start_date.date())
            return (
                StandardSnowball(
                    r=r,
                    q=q,
                    vol=vol,
                    s0=s0,
                    s=s0,
                    start_date=start_date.date(),
                    end_date=end_date.date(),
                    obs_dates=obs_dates,
                    lock_term=lock_term,
                    coupon_out=c,
                    coupon_div=c,
                    barrier_out=ko,
                    barrier_in=ki,
                    # s_steps=2000,
                ).price()
                - s0
            )

        # 理论票息取2位小数
        coupon_out = coupon_div = round(
            optimize.root_scalar(
                coupon2price, bracket=[0.0, 0.6], method="brentq"
            ).root,
            2,
        )
    else:
        coupon_out = coupon_div = coupon
    # 初始化期权类
    option = StandardSnowball(
        r=r,
        q=q,
        vol=vol,
        s0=s0,
        s=s0,
        start_date=start_date.date(),
        end_date=end_date.date(),
        obs_dates=obs_dates,
        lock_term=lock_term,
        coupon_out=coupon_out,
        coupon_div=coupon_div,
        barrier_out=ko,
        barrier_in=ki,
    )
    # 恢复logging
    logging.disable(logging.NOTSET)

    return option

回测函数

In [6]:
def backtest(
    r,
    q,
    vol,
    coupon,
    stock_df,
    stock,
    ki_ratio,
    ko_ratio,
    start_date,
    end_date,
    bt_start_date,
    bt_end_date,
    obs_dates,
    lock_term,
    buy_fee,
    sell_fee,
    cash,
    reset=True,
    back_calc=False,
) -> tuple:
    """净值化雪球回测

    Args:
        r (float): 无风险利率
        q (float): 分红融券率
        vol (float): 波动率
        index_df (pd.DataFrame): 指数价格数据
        etf_df (pd.DataFrame): ETF价格数据
        index (str): 指数代码
        etf (str): ETF代码
        ki_ratio (float): 敲入比例
        ko_ratio (float): 敲出比例
        start_date (dt.datetime): 雪球合约开始日期
        end_date (dt.datetime): 雪球合约结束日期
        bt_start_date (dt.datetime): 回测开始日期
        bt_end_date (dt.datetime): 回测结束日期
        obs_dates (list): 观察日列表
        lock_term (int): 锁定期
        buy_fee (float): 买入费率
        sell_fee (float): 卖出费率
        cash (int): 名义本金（或100%保证金）
        reset (bool, optional): 是否敲出/到期后5个交易日重置雪球合约. Defaults to True.

    Returns:
        tuple: 回测结果
    """
    s0 = stock_df.loc[start_date:].iloc[0]
    ki = s0 * ki_ratio
    ko = s0 * ko_ratio
    # 根据delta对冲获取仓位
    bt_price = stock_df.loc[bt_start_date:bt_end_date]
    option = get_option(
        r,
        q,
        vol,
        s0,
        coupon,
        start_date,
        end_date,
        obs_dates,
        lock_term,
        ko,
        ki,
        back_calc,
    )
    daily_delta = np.zeros(len(bt_price))
    # 初始化重置标志
    reset_counter = 0
    reset_date = None
    reset_date_list = []
    for i, v in enumerate(bt_price.index):
        # 更新估值日期
        set_evaluation_date(v.date())
        # 计算delta
        daily_delta[i] = option.delta(bt_price.iloc[i])
        # 判断是否敲出或到期
        if (bt_price.iloc[i] >= ko and v.date() in obs_dates) or v >= end_date:
            # 判断是否重置
            if reset is True:
                if reset_counter == 0:
                    reset_date = v
                    reset_date_list.append(reset_date)
                    reset_price = bt_price.iloc[i]
                    reset_counter += 1
            elif reset is False:
                if bt_price.iloc[i] >= ko and v.date() in obs_dates:
                    logging.info(f"{v.date()}敲出，重置雪球合约")
                elif v >= end_date:
                    logging.info(f"{v.date()}到期，重置雪球合约")
                break
        # 等待五个交易日后重置
        if reset_counter > 0:
            if reset_counter == 5:
                if reset_price >= ko and reset_date.date() in obs_dates:
                    logging.info(f"{reset_date.date()}敲出，重置雪球合约")
                elif reset_date >= end_date:
                    logging.info(f"{reset_date.date()}到期，重置雪球合约")
                start_date = v
                end_date = calendar[calendar >= start_date + rd.relativedelta(years=2)][
                    0
                ]
                vol = round(
                    index_df.pct_change(fill_method=None)
                    .rolling(252)
                    .std()
                    .loc[start_date]
                    * np.sqrt(252),
                    2,
                )
                s0 = bt_price.iloc[i]
                ki = s0 * ki_ratio
                ko = s0 * ko_ratio
                obs_dates = get_obs_dates(start_date, end_date, lock_term)
                option = get_option(
                    r,
                    q,
                    vol,
                    s0,
                    coupon,
                    start_date,
                    end_date,
                    obs_dates,
                    lock_term,
                    ko,
                    ki,
                    back_calc,
                )
                # 重置计数器和日期
                reset_counter = 0
                reset_date = None
                daily_delta[-1] = daily_delta[-2]
            else:
                reset_counter += 1
                daily_delta[-1] = daily_delta[-2]

    # 根据delta推算仓位市值
    delta_cash = pd.Series(
        cash * np.clip(np.array(daily_delta), 0, np.inf), index=bt_price.index
    )
    # 计算delta_cash对应的仓位
    amount = delta_cash / bt_price
    # 计算每日需要调整的仓位
    amount_daily = amount.diff(1)
    amount_daily.iloc[0] = amount.iloc[0]
    # 计算每日交易现金流
    trade = round(amount_daily) * bt_price
    # 计算交易费用
    fee_ratio = np.where(trade >= 0, buy_fee, sell_fee)

    # 考虑到单纯依据delta计算仓位可能会穿仓，每日重新计算仓位
    cash_left = np.empty(len(bt_price))
    # 初始化第1个交易日的数据
    amount_daily.iloc[0] = round(
        min(
            cash / (bt_price.iloc[0] * (1 + fee_ratio[0])),
            delta_cash.iloc[0] / bt_price.iloc[0],
        )
    )
    trade.iloc[0] = amount_daily.iloc[0] * bt_price.iloc[0]
    cash_left[0] = cash - trade.iloc[0] * (1 + fee_ratio[0])
    amount.iloc[0] = amount_daily.iloc[0]

    for i in range(1, len(bt_price)):
        # 限制每日仓位调整幅度
        amount_daily.iloc[i] = round(
            min(
                cash_left[i - 1] / (bt_price.iloc[i] * (1 + fee_ratio[i])),
                delta_cash.iloc[i] / bt_price.iloc[i] - amount.iloc[i - 1],
            )
        )
        trade.iloc[i] = amount_daily.iloc[i] * bt_price.iloc[i]
        cash_left[i] = cash_left[i - 1] - trade.iloc[i] * (1 + fee_ratio[i])
        # 更新当日delta_cash仓位
        amount.iloc[i] = amount.iloc[i - 1] + amount_daily.iloc[i]

    # 计算持仓市值
    snowball_positions = amount_daily.cumsum() * bt_price
    snowball_positions.rename(stock, inplace=True)
    # 计算净值变化
    snowball_returns = cash_left + snowball_positions - trade * fee_ratio
    snowball_returns.rename(stock, inplace=True)

    # 计算股票基准收益
    stock_returns = bt_price / bt_price.iloc[0]
    stock_returns.rename(stock, inplace=True)

    # 计算Gamma Scalping的基准收益
    gs_bench_returns = cash_left[0] + amount.iloc[0] * bt_price
    gs_bench_returns.rename(stock, inplace=True)

    # 回收内存
    gc.collect()

    return (
        snowball_returns,
        snowball_positions,
        stock_returns,
        gs_bench_returns,
        daily_delta,
        reset_date_list,
    )

绩效评价函数

In [7]:
def performance(pnl):
    returns = pnl.pct_change().dropna()
    annualized_return = pnl.iloc[-1] ** (252 / len(pnl)) - 1
    annualized_vol = returns.std() * np.sqrt(252)
    sharpe_ratio = annualized_return / annualized_vol
    max_drawdown = (pnl.cummax() - pnl).max()
    return {
        "年化收益率": format(annualized_return, ".2%"),
        "年化波动率": format(annualized_vol, ".2%"),
        "夏普比率": format(sharpe_ratio, ".2f"),
        "最大回撤": format(max_drawdown, ".2%"),
    }


### 回测

股票组合

In [8]:
stock_list = [
    "002993",
    "002142",
    "00902",
    "01071",
    "03800",
    "01193",
    "300751",
    "600166",
    "603197",
    "300979",
    "600129",
    "600276",
]
type_list = []
for stock in stock_list:
    # 判断为A股代码还是H股代码
    if len(stock) == 6:
        type_list.append("stock_A")
    else:
        type_list.append("stock_H")


组合回测

In [11]:
lock_term = 3
r = 0.015
q = 0.00
ki_ratio = 0.80
ko_ratio = 1.10
buy_fee = 0.0005
sell_fee = 0.0015
cash = 1e6
coupon = 0.15

snowball_returns = pd.DataFrame()
snowball_positions = pd.DataFrame()
stock_returns = pd.DataFrame()
gs_bench_returns = pd.DataFrame()

for stock, stock_type in tzip(stock_list, type_list):
    stock_df = get_price(code=stock, type=stock_type)

    calendar = stock_df.index
    calendar = calendar.append(
        pd.date_range(
            calendar[-1] + rd.relativedelta(days=1),
            calendar[-1] + rd.relativedelta(years=5),
            freq="B",
        )
    )

    start_date = calendar[calendar >= dt.datetime(2024, 4, 18)][0]
    end_date = calendar[calendar >= start_date + rd.relativedelta(years=2)][0]
    obs_dates = get_obs_dates(start_date, end_date, lock_term)

    bt_start_date = start_date
    bt_end_date = calendar[calendar <= dt.datetime.now()][-2]

    # vol = round(
    #     (stock_df.pct_change().rolling(252).std() * np.sqrt(252)).loc[start_date], 2
    # )
    vol = 0.2

    (
        snowball_returns_tmp,
        snowball_positions_tmp,
        stock_returns_tmp,
        gs_bench_returns_tmp,
        daily_delta_tmp,
        _,
    ) = backtest(
        r,
        q,
        vol,
        coupon,
        stock_df,
        stock,
        ki_ratio,
        ko_ratio,
        start_date,
        end_date,
        bt_start_date,
        bt_end_date,
        obs_dates,
        lock_term,
        buy_fee,
        sell_fee,
        cash,
        reset=True,
        back_calc=False,
    )

    snowball_returns = pd.merge(
        snowball_returns,
        snowball_returns_tmp,
        how="outer",
        left_index=True,
        right_index=True,
    )
    snowball_positions = pd.merge(
        snowball_positions,
        snowball_positions_tmp,
        how="outer",
        left_index=True,
        right_index=True,
    )
    stock_returns = pd.merge(
        stock_returns, stock_returns_tmp, how="outer", left_index=True, right_index=True
    )
    gs_bench_returns = pd.merge(
        gs_bench_returns,
        gs_bench_returns_tmp,
        how="outer",
        left_index=True,
        right_index=True,
    )

snowball_returns.sort_index(inplace=True)
snowball_returns.ffill(inplace=True)
snowball_positions.sort_index(inplace=True)
snowball_positions.ffill(inplace=True)
stock_returns.sort_index(inplace=True)
stock_returns.ffill(inplace=True)
gs_bench_returns.sort_index(inplace=True)
gs_bench_returns.ffill(inplace=True)
gs_bench_returns.ffill(inplace=True)

snowball_return = snowball_returns.sum(axis=1) / (12 * 1e6)
bench_return = stock_returns.mean(axis=1)
gs_bench_return = gs_bench_returns.sum(axis=1) / (12 * 1e6)
exceed_return = snowball_return - bench_return
gs_exceed_return = snowball_return - gs_bench_return
portfolio_position = snowball_positions.sum(axis=1) / (12 * 1e6)

  0%|          | 0/12 [00:00<?, ?it/s]

2024-07-12 09:38:48,744 - INFO: price执行完成，耗时1.815秒
2024-07-12 09:38:50,657 - INFO: price执行完成，耗时1.912秒
2024-07-12 09:38:53,552 - INFO: price执行完成，耗时1.882秒
2024-07-12 09:38:55,382 - INFO: price执行完成，耗时1.829秒
2024-07-12 09:38:58,103 - INFO: price执行完成，耗时1.824秒
2024-07-12 09:38:59,960 - INFO: price执行完成，耗时1.856秒
2024-07-12 09:39:02,555 - INFO: price执行完成，耗时1.778秒
2024-07-12 09:39:04,275 - INFO: price执行完成，耗时1.719秒
2024-07-12 09:39:06,852 - INFO: price执行完成，耗时1.820秒
2024-07-12 09:39:08,651 - INFO: price执行完成，耗时1.798秒
2024-07-12 09:39:11,253 - INFO: price执行完成，耗时1.750秒
2024-07-12 09:39:13,040 - INFO: price执行完成，耗时1.786秒
2024-07-12 09:39:15,354 - INFO: price执行完成，耗时1.703秒
2024-07-12 09:39:17,133 - INFO: price执行完成，耗时1.778秒
2024-07-12 09:39:19,538 - INFO: price执行完成，耗时1.664秒
2024-07-12 09:39:21,162 - INFO: price执行完成，耗时1.623秒
2024-07-12 09:39:23,311 - INFO: price执行完成，耗时1.489秒
2024-07-12 09:39:24,916 - INFO: price执行完成，耗时1.604秒
2024-07-12 09:39:27,135 - INFO: price执行完成，耗时1.644秒
2024-07-12 09:39:28,866 - INFO:

绘制回测曲线和每日Delta

In [12]:
# 创建图表
fig = make_subplots(
    rows=2,
    cols=1,
    subplot_titles=[
        "每日净值",
        "每日Delta",
    ],
    specs=[[{"secondary_y": True}], [{}]],  # 在第一行添加secondary_y以支持右侧y轴
)

# 添加线图
fig.add_trace(
    go.Scatter(
        x=snowball_return.index,
        y=snowball_return,
        mode="lines",
        name="净值化雪球净值",
        legendgroup="1",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=bench_return.index,
        y=bench_return,
        mode="lines",
        name="股票基准净值",
        legendgroup="1",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=exceed_return.index,
        y=exceed_return,
        mode="lines",
        name="超额收益",
        legendgroup="1",
    ),
    row=1,
    col=1,
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(
        x=portfolio_position.index,
        y=portfolio_position,
        mode="lines",
        name="每日仓位",
        legendgroup="2",
    ),
    row=2,
    col=1,
)

fig["layout"]["xaxis"].update({"title": "日期", "tickformat": "%Y-%m-%d"})
fig["layout"]["yaxis"].update({"title": "净值", "tickformat": ".2f"})
fig["layout"]["xaxis2"].update({"title": "日期", "tickformat": "%Y-%m-%d"})
fig["layout"]["yaxis2"].update({"title": "超额收益", "tickformat": ".2%"})
fig["layout"]["yaxis3"].update({"title": "Delta", "tickformat": ".2%"})
# 设置图表布局
fig.update_layout(
    template="plotly_white",
    height=800,
    width=1000,
    hovermode="x unified",  # 设置鼠标悬停模式
    legend_tracegroupgap=330,
)

fig.show()

回测绩效评价

In [13]:
pd.DataFrame(
    [
        performance(snowball_return),
        performance(bench_return),
        performance(gs_bench_return),
    ],
    index=["净值化雪球", "股票基准", "Gamma Scalping基准"],
)

,年化收益率,年化波动率,夏普比率,最大回撤
净值化雪球,2.54%,7.71%,0.33,5.29%
股票基准,-7.04%,16.37%,-0.43,12.63%
Gamma Scalping基准,-3.71%,8.40%,-0.44,6.36%


In [14]:
his_vol = []
realized_vol = []
for stock, stock_type in tzip(stock_list, type_list):
    stock_df = get_price(code=stock, type=stock_type)
    calendar = stock_df.index
    start_date = calendar[calendar >= dt.datetime(2024, 4, 18)][0]
    end_date = calendar[calendar <= dt.datetime.now()][-1]
    his_vol.append(
        stock_df.pct_change()
        .rolling(len(stock_df.loc[start_date:end_date]))
        .std()
        .loc[start_date]
        * np.sqrt(252)
    )
    realized_vol.append(
        stock_df.pct_change().loc[start_date:end_date].std() * np.sqrt(252)
    )
pd.DataFrame({"HV": his_vol, "RV": realized_vol}, index=stock_list)


  0%|          | 0/12 [00:00<?, ?it/s]

,HV,RV
002993,0.563647,0.332371
002142,0.320000,0.258502
00902,0.456979,0.358336
01071,0.427910,0.501732
03800,0.599333,0.537479
01193,0.448144,0.344280
300751,0.467046,0.449559
600166,0.428130,0.244662
603197,0.533839,0.463747
300979,0.353352,0.310903
